## 서울 열린 데이터 광장 홈페이지에서 공공데이터 목록들을 수집해보자.

In [2]:
import requests as req
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

In [4]:
U_A = {'User-Agent':"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36"}

In [178]:
url = 'https://data.seoul.go.kr/dataList/datasetList.do'
res_seoul = req.get(url, headers = U_A)
res_seoul

<Response [200]>

In [18]:
soup_seoul = bs(res_seoul.text, 'lxml')
# lxml: C언어로 작성된 parser로 큰 문서를 다룰 때 속도가 빨라서 유리하며 HTML 구조를 잘 보정하여 파싱해줌.
# XML에 대한 파싱도 지원함.
# html.parser : 파이썬 표준 라이브러리에 내장된 HTML parser로 상대적으로 느리면 보정력이 좋지 않음. XML 파싱도 안 해줌.
soup_seoul

<!DOCTYPE html>
<html lang="ko">
<head>
<!-- Global site tag (gtag.js) Google Analytics -->
<script async="" src="//www.googletagmanager.com/gtag/js?id=G-0T3XG23CN7"></script>
<script>
		window.dataLayer = window.dataLayer || [];
		function gtag(){dataLayer.push(arguments);}
		gtag('js', new Date());
		gtag('config', 'G-0T3XG23CN7');
	</script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="title"/>
<meta content="" name="keywords"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<!-- meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no"/--><!-- ios때문에 이렇게 한다고하는데..w3s의 권장은 윗줄 -->
<meta content="website" property="og:type"/>
<meta content="" property="og:title"/>
<meta content="" property="og:description"/>
<meta content="https://data.seoul.go.kr:8580/resources/img/common/logo.pn

### 데이터셋의 제목, 분류, 제공기관, 수정일자 수집

In [95]:
title = soup_seoul.select('.goView strong')
title

[<strong>서울시 중구 위생처리업 공중위생업소 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 기타 위생용품 제조업 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 위생처리업 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 세척제 제조업 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울글로벌센터 접수유형 및 상담유형별(월) 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>상상대로 서울 자유제안 정보 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 하수관로 수위 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울글로벌센터 상담유형별(월) 상담실적 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울시 위생업소 전체 행정처분내역 현황 <img alt="업데이트" src="../../resources/img/common/icon_Update.png"/></strong>,
 <strong>서울글로벌센터 상담유형별(분기) 상담실적 <img alt="업데이트" src=

In [149]:
category = soup_seoul.select('#datasetVO > div.wrap-a > div > section > div.list-statistics > dl > dt > em:nth-child(2)')
# category = category[1::2] 이렇게 해도 답 나옴.
for i in category:
    print(i.text) 

[보건]
[보건]
[보건]
[보건]
[복지]
[일반행정]
[도시관리]
[복지]
[보건]
[복지]


In [137]:
edit_date = soup_seoul.select('.list-statistics-info2 > span:nth-child(1)')

edit_date_list = []
for i in edit_date:
    date_strip = i.text.strip()[-10:]
    edit_date_list.append(date_strip)

edit_date_list

['2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10',
 '2025-07-10']

In [171]:
provider = soup_seoul.select('.list-statistics-info2 > span:nth-child(3)')

print(len("제공부서 : \
										"))

provider_list = []
for i in provider:
    pro_strip = i.text.strip()[17:]
    provider_list.append(pro_strip.strip())

provider_list

17


['디지털도시국 정보시스템과',
 '시민건강국 감염병관리과',
 '디지털도시국 데이터전략과',
 '디지털도시국 데이터전략과',
 '경제실 경제일자리기획관 금융투자과',
 '홍보기획관 콘텐츠담당관',
 '물순환안전국 물재생계획과',
 '경제실 경제일자리기획관 금융투자과',
 '디지털도시국 데이터전략과',
 '경제실 경제일자리기획관 금융투자과']

### GET, POST 통신 방식
- 두 방식 모두 HTTP 내에서 클라이언트가 서버에 데이터를 요청하거나 전송할 때 사용
- GET은 데이터가 URL에 노출됨.(데이터의 길이 제한이 있으면 일반적으로 2048자까지 제한)
- POST는 GET보다 보안이 높은 방식으로 데이터가 URL에 노출되지 않음.(전송 데이터의 종류 및 크기 제한이 없음.)

In [188]:
payload = {'pageIndex':1}

# post: POST방식으로 요청 보내기
# data인자는 폼 데이터를 설정하는 매개변수로 딕셔너리로 설정가능하며 웹 접근 시 제출해야하는 정보를 포함함
res_seoul = req.post(url, data=payload, headers = U_A)
res_seoul

<Response [200]>

In [190]:
soup_seoul = bs(res_seoul.text, 'lxml')
soup_seoul

<!DOCTYPE html>
<html lang="ko">
<head>
<!-- Global site tag (gtag.js) Google Analytics -->
<script async="" src="//www.googletagmanager.com/gtag/js?id=G-0T3XG23CN7"></script>
<script>
		window.dataLayer = window.dataLayer || [];
		function gtag(){dataLayer.push(arguments);}
		gtag('js', new Date());
		gtag('config', 'G-0T3XG23CN7');
	</script>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="" name="title"/>
<meta content="" name="keywords"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<!-- meta name="viewport" content="width=device-width,initial-scale=1.0,minimum-scale=1.0,maximum-scale=1.0,user-scalable=no"/--><!-- ios때문에 이렇게 한다고하는데..w3s의 권장은 윗줄 -->
<meta content="website" property="og:type"/>
<meta content="" property="og:title"/>
<meta content="" property="og:description"/>
<meta content="https://data.seoul.go.kr:8580/resources/img/common/logo.pn

In [202]:
title_list, category_list, edit_date_list, provider_list = [], [], [], []

for i in range(1,6):
    payload = {'pageIndex': i}
    res_seoul = req.post(url, data=payload, headers = U_A)
    soup_seoul = bs(res_seoul.text, 'lxml')
    
    title = soup_seoul.select('.goView strong')
    for i in title:
        title_list.append(i.text.strip())
    
    category = soup_seoul.select('#datasetVO > div.wrap-a > div > section > div.list-statistics > dl > dt > em:nth-child(2)')
    for i in category:
        category_list.append(i.text)
    
    edit_date = soup_seoul.select('.list-statistics-info2 > span:nth-child(1)')
    for i in edit_date:
        date_strip = i.text.strip()[-10:]
        edit_date_list.append(date_strip)

    provider = soup_seoul.select('.list-statistics-info2 > span:nth-child(3)')
    for i in provider:
        pro_strip = i.text.strip()[17:]
        provider_list.append(pro_strip.strip())

print(title_list, len(title_list))
print(category_list, len(category_list))
print(edit_date_list, len(edit_date_list))
print(provider_list, len(provider_list))

['서울시 기타 위생용품 제조업 현황', '서울시 중구 위생처리업 공중위생업소 현황', '서울시 세척제 제조업 현황', '서울시 위생처리업 현황', '서울글로벌센터 접수유형 및 상담유형별(월) 상담실적', '상상대로 서울 자유제안 정보', '서울글로벌센터 상담유형별(월) 상담실적', '서울시 위생업소 전체 행정처분내역 현황', '서울글로벌센터 상담유형별(분기) 상담실적', '서울글로벌센터 접수유형별 언어별 상담실적', '서울시 집수리닷컴 공지사항', '서울시 집수리 시공업체 정보', '집수리 아카데미 수료자 활동내역', '서울글로벌센터 언어별(월) 상담실적', '집수리 성공기', '집수리 성공기 첨부내용', '집수리 아카데미 교육정보', '서울시 어르신 취업지원센터 교육정보(~2024년)', '서울시 대기오염 측정소 정보', '서울시 문화공간 정보', '서울시 한옥비용지원 내역 정보', '서울시 작은도서관 현황정보', '서울도서관 일정 정보', '서울시 문화행사 정보', '서울시 도서관 이용시간 및 휴관일 정보', '서울도서관 새로 들어온 도서정보', '서울시 장애인 도서관 현황정보', '서울도서관 강좌정보', '서울시 공공도서관 현황정보', '서울도서관 소장자료 현황정보', '서울시 대기오염 측정항목 정보', '서울시 부동산 중개업소 정보', '서울시 청소년수련관 강좌정보', '서울시 한옥등록 정보', '서울시 염화칼슘보관함 위치정보', '서울시 공공자전거 신규가입자 정보(월별)', '한강공원 주차장 요금감면 현황', '한강공원 주차장 연별 이용 현황', '한강공원 주차장 일별 이용 현황', '한강공원 주차장 월별 이용 현황', '서울시 대여 공구 찾기 정보', '서울시 공유 기업 정보', '한강공원 주차장 정보', '서울시 우리동네 공구대여소 정보', '서울시 야경명소 정보', '서울시 부문별 지출금액 총액 정보', '서울시 세출운용 사업 및 예산 정보', '서울시 세입운용 현황', '서울시 일자별 지출금액 (전체) 정보', '서울시 일별 지출 총액 정보'

In [206]:
seoul_dic = {'dataset title':title_list,'category':category_list,'edit date': edit_date_list,'data provider':provider_list}
seoul_df = pd.DataFrame(seoul_dic, index = range(1,len(title_list)+1))
seoul_df

,dataset title,category,edit date,data provider
1,서울시 기타 위생용품 제조업 현황,[보건],2025-07-10,시민건강국 감염병관리과
2,서울시 중구 위생처리업 공중위생업소 현황,[보건],2025-07-10,디지털도시국 정보시스템과
3,서울시 세척제 제조업 현황,[보건],2025-07-10,디지털도시국 데이터전략과
4,서울시 위생처리업 현황,[보건],2025-07-10,디지털도시국 데이터전략과
5,서울글로벌센터 접수유형 및 상담유형별(월) 상담실적,[복지],2025-07-10,경제실 경제일자리기획관 금융투자과
6,상상대로 서울 자유제안 정보,[일반행정],2025-07-10,홍보기획관 콘텐츠담당관
7,서울글로벌센터 상담유형별(월) 상담실적,[복지],2025-07-10,경제실 경제일자리기획관 금융투자과
8,서울시 위생업소 전체 행정처분내역 현황,[보건],2025-07-10,디지털도시국 데이터전략과
9,서울글로벌센터 상담유형별(분기) 상담실적,[복지],2025-07-10,경제실 경제일자리기획관 금융투자과
10,서울글로벌센터 접수유형별 언어별 상담실적,[복지],2025-07-10,경제실 경제일자리기획관 금융투자과
